In [1]:
###Yield Management:

In [2]:
!pip install gurobipy



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\olw09\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import gurobipy as gp
from gurobipy import GRB

# Define the lines for the tic-tac-toe constraints
lines = []
size = 3

for i in range(size):
    for j in range(size):
        for k in range(size):
            if i == 0:
                lines.append(((0,j,k), (1,j,k), (2,j,k)))
            if j == 0:
                lines.append(((i,0,k), (i,1,k), (i,2,k)))
            if k == 0:
                lines.append(((i,j,0), (i,j,1), (i,j,2)))
            if i == 0 and j == 0:
                lines.append(((0,0,k), (1,1,k), (2,2,k)))
            if i == 0 and j == 2:
                lines.append(((0,2,k), (1,1,k), (2,0,k)))
            if i == 0 and k == 0:
                lines.append(((0,j,0), (1,j,1), (2,j,2)))
            if i == 0 and k == 2:
                lines.append(((0,j,2), (1,j,1), (2,j,0)))
            if j == 0 and k == 0:
                lines.append(((i,0,0), (i,1,1), (i,2,2)))
            if j == 0 and k == 2:
                lines.append(((i,0,2), (i,1,1), (i,2,0)))

# Add the diagonal lines
lines.append(((0,0,0), (1,1,1), (2,2,2)))
lines.append(((2,0,0), (1,1,1), (0,2,2)))
lines.append(((0,2,0), (1,1,1), (2,0,2)))
lines.append(((0,0,2), (1,1,1), (2,2,0)))

print(len(lines))

model = gp.Model("tic-tac")

visit = model.addVars(size, size, size, vtype=GRB.BINARY, name='visit')
sameline = model.addVars(len(lines), vtype=GRB.BINARY, name='sameline')
model.addConstr(sum(visit[i,j,k] for i in range(size) for j in range(size) for k in range(size)) == 14)

for i, x in enumerate(lines):
    model.addConstr(sameline[i] >= visit[x[0]] + visit[x[1]] + visit[x[2]] - 2, name=f'constraint_{i}')
    model.addConstr(sameline[i] >= 1-(visit[x[0]] + visit[x[1]] + visit[x[2]]), name=f'constraint_{i}')

model.setObjective(sum(sameline[i] for i in range(len(lines))), GRB.MINIMIZE)

model.optimize()

if model.status == GRB.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {model.objVal}')
else:
    print('No optimal solution found')


49
Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 99 rows, 76 columns and 419 nonzeros
Model fingerprint: 0xf7452865
Variable types: 0 continuous, 76 integer (76 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 7.0000000
Presolve time: 0.00s
Presolved: 99 rows, 76 columns, 419 nonzeros
Variable types: 0 continuous, 76 integer (76 binary)

Root relaxation: objective 0.000000e+00, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

In [4]:
import gurobipy as gp
from gurobipy import GRB

# Data
shifts = {
    "Mon1":  3,
    "Tue2":  2,
    "Wed3":  4,
    "Thu4":  4,
    "Fri5":  5,
    "Sat6":  6,
    "Sun7":  5,
    "Mon8":  2,
    "Tue9":  2,
    "Wed10": 3,
    "Thu11": 4,
    "Fri12": 6,
    "Sat13": 7,
    "Sun14": 5
}

shiftRequirements = {
    "Mon1":  3,
    "Tue2":  2,
    "Wed3":  4,
    "Thu4":  4,
    "Fri5":  5,
    "Sat6":  6,
    "Sun7":  5,
    "Mon8":  2,
    "Tue9":  2,
    "Wed10": 3,
    "Thu11": 4,
    "Fri12": 6,
    "Sat13": 7,
    "Sun14": 5
}

workers = {
    "Amy":   10,
    "Bob":   12,
    "Cathy": 10,
    "Dan":   8,
    "Ed":    8,
    "Fred":  9,
    "Gu":    11
}

pay = {
    "Amy":   10,
    "Bob":   12,
    "Cathy": 10,
    "Dan":   8,
    "Ed":    8,
    "Fred":  9,
    "Gu":    11
}

availability = [
    ('Amy', 'Tue2'), ('Amy', 'Wed3'), ('Amy', 'Fri5'), ('Amy', 'Sun7'),
    ('Amy', 'Tue9'), ('Amy', 'Wed10'), ('Amy', 'Thu11'), ('Amy', 'Fri12'),
    ('Amy', 'Sat13'), ('Amy', 'Sun14'), ('Bob', 'Mon1'), ('Bob', 'Tue2'),
    ('Bob', 'Fri5'), ('Bob', 'Sat6'), ('Bob', 'Mon8'), ('Bob', 'Thu11'),
    ('Bob', 'Sat13'), ('Cathy', 'Wed3'), ('Cathy', 'Thu4'), ('Cathy', 'Fri5'),
    ('Cathy', 'Sun7'), ('Cathy', 'Mon8'), ('Cathy', 'Tue9'), ('Cathy', 'Wed10'),
    ('Cathy', 'Thu11'), ('Cathy', 'Fri12'), ('Cathy', 'Sat13'),
    ('Cathy', 'Sun14'), ('Dan', 'Tue2'), ('Dan', 'Wed3'), ('Dan', 'Fri5'),
    ('Dan', 'Sat6'), ('Dan', 'Mon8'), ('Dan', 'Tue9'), ('Dan', 'Wed10'),
    ('Dan', 'Thu11'), ('Dan', 'Fri12'), ('Dan', 'Sat13'), ('Dan', 'Sun14'),
    ('Ed', 'Mon1'), ('Ed', 'Tue2'), ('Ed', 'Wed3'), ('Ed', 'Thu4'),
    ('Ed', 'Fri5'), ('Ed', 'Sun7'), ('Ed', 'Mon8'), ('Ed', 'Tue9'),
    ('Ed', 'Thu11'), ('Ed', 'Sat13'), ('Ed', 'Sun14'), ('Fred', 'Mon1'),
    ('Fred', 'Tue2'), ('Fred', 'Wed3'), ('Fred', 'Sat6'), ('Fred', 'Mon8'),
    ('Fred', 'Tue9'), ('Fred', 'Fri12'), ('Fred', 'Sat13'), ('Fred', 'Sun14'),
    ('Gu', 'Mon1'), ('Gu', 'Tue2'), ('Gu', 'Wed3'), ('Gu', 'Fri5'),
    ('Gu', 'Sat6'), ('Gu', 'Sun7'), ('Gu', 'Mon8'), ('Gu', 'Tue9'),
    ('Gu', 'Wed10'), ('Gu', 'Thu11'), ('Gu', 'Fri12'), ('Gu', 'Sat13'),
    ('Gu', 'Sun14')
]

m = gp.Model("shift_scheduling")

working = []
slack = []

for i, shift in enumerate(shifts):
    working.append([])
    var1 = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f'shift{i}')
    slack.append(var1)
    for j, worker in enumerate(workers):
        var = m.addVar(vtype=GRB.BINARY, name=f'worker{j}_shift{i}')
        working[-1].append(var)

m.update()

for i, shift in enumerate(shifts):
    m.addConstr(sum(working[i]) + slack[i] == shiftRequirements[shift])
    for j, worker in enumerate(workers):
        if (worker, shift) not in availability:
            m.addConstr(working[i][j] == 0)
max_workload = m.addVar(name="max_workload")
min_workload = m.addVar(name="min_workload")
for j, worker in enumerate(workers):
    workload = sum(working[i][j] for i in range(len(shifts)))
    m.addConstr(max_workload >= workload)
    m.addConstr(min_workload <= workload)


m.setObjectiveN(sum(slack), index=0,priority=2,name='adhbhv')
m.setObjectiveN(max_workload-min_workload,index=1,priority=1,name='hda')

m.optimize()

if m.status == GRB.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {m.objVal}')
    for i, shift in enumerate(shifts):
        for j, worker in enumerate(workers):
            if working[i][j].X > 0.5:
                print(f'{worker} works on {shift}')
else:
    print('No optimal solution found.')


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 54 rows, 114 columns and 348 nonzeros
Model fingerprint: 0x6d5e47fd
Variable types: 16 continuous, 98 integer (98 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 7e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 31 rows and 31 columns
Presolve time: 0.00s
Presolved: 23 rows, 83 columns, 217 nonzeros
-----

In [25]:
import json
import gurobipy as gp
from gurobipy import GRB
import math
from itertools import combinations
with open('capitals.json') as f:
    capitals_json = json.load(f)

capitals = []
coordinates = {}
for state in capitals_json:
    #if state not in ['AK', 'HI']:
        capital = capitals_json[state]['capital']
        capitals.append(capital)
        coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))
def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0] - c2[0], c1[1] - c2[1])
    return math.sqrt(diff[0]**2 + diff[1]**2)
dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}

model=gp.Model()
vars = model.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

vars.update({(j,i):vars[i,j] for i,j in vars.keys()})

cons = model.addConstrs(vars.sum(c, '*') == 2 for c in capitals)
def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:]
    while unvisited:
        this = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            this.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(this) < len(cycle):
            cycle = this
    return cycle
def callbacki(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys() if vals[i, j] > 0.5)
        tour = subtour(selected)
        if len(tour) < len(capitals):
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2)) <= len(tour) - 1)

model._vars = vars
model.Params.lazyConstraints = 1
model.optimize(callbacki)



vals = model.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)
tour = subtour(selected)
assert len(tour) == len(capitals)
print(tour)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 50 rows, 1225 columns and 2450 nonzeros
Model fingerprint: 0x97d625ab
Variable types: 0 continuous, 1225 integer (1225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 1119.8692996
Presolve time: 0.01s
Presolved: 50 rows, 1225 columns, 2450 nonzeros
Variable types: 0 continuous, 1225 integer (1225 binary)

Root relaxation: objective 2.535434e+02, 72 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [23]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()

TASKS = {
    ('paper_1','Blue')   : {'dur': 45, 'prec': None},
    ('paper_1','Yellow') : {'dur': 10, 'prec': ('paper_1','Blue')},
    ('paper_2','Blue')   : {'dur': 20, 'prec': ('paper_2','Green')},
    ('paper_2','Green')  : {'dur': 10, 'prec': None},
    ('paper_2','Yellow') : {'dur': 34, 'prec': ('paper_2','Blue')},
    ('paper_3','Blue')   : {'dur': 12, 'prec': ('paper_3','Yellow')},
    ('paper_3','Green')  : {'dur': 17, 'prec': ('paper_3','Blue')},
    ('paper_3','Yellow') : {'dur': 28, 'prec': None},   
}

papers = ['paper_1', 'paper_2', 'paper_3']
jobs = ['Blue', 'Yellow', 'Green']

makespan = solver.NumVar(0, solver.infinity(), 'makespan')
start = {}

order_job={}
for i,job in enumerate(jobs):
    for j1,paper1 in enumerate(papers):
        for j2,paper2 in enumerate(papers):
            if (paper1,job) in TASKS and (paper2,job) in TASKS and paper1!=paper2:
                order_job[(i,j1,j2)]=solver.BoolVar(f'order_{job}_{paper1}_before_{paper2}')

for i, paper in enumerate(papers):
    for j, job in enumerate(jobs):
        if (paper, job) in TASKS:
            start[(i, j)] = solver.NumVar(0, solver.infinity(), f'start_{paper}_{job}')
            
           
for i, paper in enumerate(papers):
    for j, job in enumerate(jobs):
        if (paper, job) in TASKS:
            solver.Add(start[(i, j)] + TASKS[(paper, job)]['dur'] <= makespan)
            prec = TASKS[(paper, job)]['prec']
            if prec is not None:
                k = papers.index(prec[0])
                l = jobs.index(prec[1])
                if (prec[0], prec[1]) in TASKS:
                    solver.Add(start[(k, l)] + TASKS[(prec[0], prec[1])]['dur'] <= start[(i, j)])


for i, job in enumerate(jobs):
    for j1, paper1 in enumerate(papers):
        for j2, paper2 in enumerate(papers):
            if (paper1, job) in TASKS and (paper2, job) in TASKS and j1 != j2:
                order_var = order_job[(i, j1, j2)]
                solver.Add(start[(j1, i)] + TASKS[(paper1, job)]['dur'] <= start[(j2, i)] + (1 - order_var) * 176)
                solver.Add(start[(j2, i)] + TASKS[(paper2, job)]['dur'] <= start[(j1, i)] + order_var * 176)


solver.Minimize(makespan)
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())

    schedule = {job: [] for job in jobs}
    
    for i, paper in enumerate(papers):
        for j, job in enumerate(jobs):
            if (paper, job) in TASKS:
                start_time = start[(i, j)].solution_value()
                duration = TASKS[(paper, job)]['dur']
                finish_time = start_time + duration
                schedule[job].append((paper, start_time, duration, finish_time))

    for job in jobs:
        print(f"{job:<7} Paper   Start  Duration  Finish")
        for entry in sorted(schedule[job], key=lambda x: x[1]):
            paper, start_time, duration, finish_time = entry
            print(f"{'':<7} {paper:<7} {start_time:>5.1f} {duration:>8} {finish_time:>7.1f}")

else:
    print('No optimal solution found.')

Solution:
Objective value = 96.99999999999999
Blue    Paper   Start  Duration  Finish
        paper_2  10.0       20    30.0
        paper_3  30.0       12    42.0
        paper_1  42.0       45    87.0
Yellow  Paper   Start  Duration  Finish
        paper_3   0.0       28    28.0
        paper_2  53.0       34    87.0
        paper_1  87.0       10    97.0
Green   Paper   Start  Duration  Finish
        paper_2   0.0       10    10.0
        paper_3  80.0       17    97.0


In [24]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print('Solver not created.')
    exit()

TASKS = {
    ('A', 'Mixer'): {'dur': 1, 'prec': None},
('A', 'Reactor'): {'dur': 5, 'prec': ('A', 'Mixer')},
('A', 'Separator'): {'dur': 4, 'prec': ('A', 'Reactor')},
('A', 'Packaging'): {'dur': 1.5, 'prec': ('A', 'Separator')},
('B', 'Separator'): {'dur': 4.5, 'prec': None},
('B', 'Packaging'): {'dur': 1, 'prec': ('B', 'Separator')},
('C', 'Separator') :{'dur': 5, 'prec': None},
('C', 'Reactor'): {'dur': 3, 'prec': ('C', 'Separator')},
('C', 'Packaging') :{'dur': 1.5, 'prec': ('C', 'Reactor')}
}

papers = ['A', 'B', 'C']
jobs = ['Mixer', 'Reactor','Separator','Packaging']

makespan = solver.NumVar(0, solver.infinity(), 'makespan')
start = {}

order_job={}
for i,job in enumerate(jobs):
    for j1,paper1 in enumerate(papers):
        for j2,paper2 in enumerate(papers):
            if (paper1,job) in TASKS and (paper2,job) in TASKS and paper1!=paper2:
                order_job[(i,j1,j2)]=solver.BoolVar(f'order_{job}_{paper1}_before_{paper2}')

for i, paper in enumerate(papers):
    for j, job in enumerate(jobs):
        if (paper, job) in TASKS:
            start[(i, j)] = solver.NumVar(0, solver.infinity(), f'start_{paper}_{job}')
            
           
for i, paper in enumerate(papers):
    for j, job in enumerate(jobs):
        if (paper, job) in TASKS:
            solver.Add(start[(i, j)] + TASKS[(paper, job)]['dur'] <= makespan)
            prec = TASKS[(paper, job)]['prec']
            if prec is not None:
                k = papers.index(prec[0])
                l = jobs.index(prec[1])
                if (prec[0], prec[1]) in TASKS:
                    solver.Add(start[(k, l)] + TASKS[(prec[0], prec[1])]['dur'] <= start[(i, j)])


for i, job in enumerate(jobs):
    for j1, paper1 in enumerate(papers):
        for j2, paper2 in enumerate(papers):
            if (paper1, job) in TASKS and (paper2, job) in TASKS and j1 != j2:
                order_var = order_job[(i, j1, j2)]
                solver.Add(start[(j1, i)] + TASKS[(paper1, job)]['dur'] <= start[(j2, i)] + (1 - order_var) * 176)
                solver.Add(start[(j2, i)] + TASKS[(paper2, job)]['dur'] <= start[(j1, i)] + order_var * 176)


solver.Minimize(makespan)
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())

    schedule = {job: [] for job in jobs}
    
    for i, paper in enumerate(papers):
        for j, job in enumerate(jobs):
            if (paper, job) in TASKS:
                start_time = start[(i, j)].solution_value()
                duration = TASKS[(paper, job)]['dur']
                finish_time = start_time + duration
                schedule[job].append((paper, start_time, duration, finish_time))

    for job in jobs:
        print(f"{job:<7} Paper   Start  Duration  Finish")
        for entry in sorted(schedule[job], key=lambda x: x[1]):
            paper, start_time, duration, finish_time = entry
            print(f"{'':<7} {paper:<7} {start_time:>5.1f} {duration:>8} {finish_time:>7.1f}")

else:
    print('No optimal solution found.')

Solution:
Objective value = 14.999999999999961
Mixer   Paper   Start  Duration  Finish
        A         0.0        1     1.0
Reactor Paper   Start  Duration  Finish
        A         1.0        5     6.0
        C         6.0        3     9.0
Separator Paper   Start  Duration  Finish
        C         0.0        5     5.0
        B         5.0      4.5     9.5
        A         9.5        4    13.5
Packaging Paper   Start  Duration  Finish
        C         9.0      1.5    10.5
        B        10.5        1    11.5
        A        13.5      1.5    15.0
